In [1]:
# go into edge-attribution-patching directory
import os

print("Current working directory: {0}".format(os.getcwd()))
os.chdir("acdcpp/ioi_task")
print("Current working directory: {0}".format(os.getcwd()))

Current working directory: /data/phillip_guo/mechanistic-unlearning
Current working directory: /data/phillip_guo/mechanistic-unlearning/acdcpp/ioi_task


In [2]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append('../Automatic-Circuit-Discovery/')
sys.path.append('../tracr/')
sys.path.append('..')
import re

import acdc
from acdc.TLACDCExperiment import TLACDCExperiment
from acdc.acdc_utils import TorchIndex, EdgeType
import numpy as np
import torch as t
from torch import Tensor
import einops
import itertools

from transformer_lens import HookedTransformer, ActivationCache

import tqdm.notebook as tqdm
import plotly
from rich import print as rprint
from rich.table import Table

from jaxtyping import Float, Bool
from typing import Callable, Tuple, Union, Dict, Optional

device = t.device('cuda') if t.cuda.is_available() else t.device('cpu')
print(f'Device: {device}')

Device: cuda


# Model Setup

In [3]:
model = HookedTransformer.from_pretrained(
    'gpt2-small',
    center_writing_weights=False,
    center_unembed=False,
    fold_ln=False,
    device=device,
)
model.set_use_hook_mlp_in(True)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)

Loaded pretrained model gpt2-small into HookedTransformer


from acdc.ioi.utils import get_ioi_true_edges
ioi_true_edges = get_ioi_true_edges(model)
for (b1, i1, b2, i2), val in ioi_true_edges.items():
    if val:
        print(b1, b2)

# Dataset Setup

In [4]:
from ioi_dataset import IOIDataset, format_prompt, make_table
N = 100
clean_dataset = IOIDataset(
    # prompt_type='mixed',
    N=N,
    tokenizer=model.tokenizer,
    prepend_bos=False,
    seed=1,
    device=device,
    prompt_type='ABBA',
    nb_templates=1
)
corr_dataset = clean_dataset.gen_flipped_prompts('ABC->XYZ, BAB->XYZ')

make_table(
  colnames = ["IOI prompt", "IOI subj", "IOI indirect obj", "ABC prompt"],
  cols = [
    map(format_prompt, clean_dataset.sentences),
    model.to_string(clean_dataset.s_tokenIDs).split(),
    model.to_string(clean_dataset.io_tokenIDs).split(),
    map(format_prompt, clean_dataset.sentences),
  ],
  title = "Sentences from IOI vs ABC distribution",
)

                                      Sentences from IOI vs ABC distribution                                       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ IOI prompt                              ┃ IOI subj ┃ IOI indirect obj ┃ ABC prompt                              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Then, Marco and William went to the     │ William  │ Marco            │ Then, Marco and William went to the     │
│ garden. William gave a computer to      │          │                  │ garden. William gave a computer to      │
│ Marco                                   │          │                  │ Marco                                   │
│                                         │          │                  │                                         │
│ Then, Kevin and Scott went to the       │ Scott    │ Kevin            │ Then, Kevin and Scott went to the       │
│ station. Scott gave a drink to Kevin    │          │                  │ station. Scott gave a drink to Kevin    │
│                                         │          │                  │                                         │
│ Then, Arthur and Kelly went to the      │ Kelly    │ Arthur           │ Then, Arthur and Kelly went to the      │
│ house. Kelly gave a drink to Arthur     │          │                  │ house. Kelly gave a drink to Arthur     │
│                                         │          │                  │                                         │
│ Then, River and Jordan went to the      │ Jordan   │ River            │ Then, River and Jordan went to the      │
│ school. Jordan gave a kiss to River     │          │                  │ school. Jordan gave a kiss to River     │
│                                         │          │                  │                                         │
│ Then, Patrick and Louis went to the     │ Louis    │ Patrick          │ Then, Patrick and Louis went to the     │
│ house. Louis gave a basketball to       │          │                  │ house. Louis gave a basketball to       │
│ Patrick                                 │          │                  │ Patrick                                 │
│                                         │          │                  │                                         │
└─────────────────────────────────────────┴──────────┴──────────────────┴─────────────────────────────────────────┘

# Metric Setup

In [5]:
def ave_logit_diff(
    logits: Float[Tensor, 'batch seq d_vocab'],
    ioi_dataset: IOIDataset,
    per_prompt: bool = False
):
    '''
        Return average logit difference between correct and incorrect answers
    '''
    # Get logits for indirect objects
    print(f"{logits.shape=}, {ioi_dataset.word_idx['end'].shape=}, {len(ioi_dataset.io_tokenIDs)=}")
    io_logits = logits[range(logits.size(0)), ioi_dataset.word_idx['end'], ioi_dataset.io_tokenIDs]
    s_logits = logits[range(logits.size(0)), ioi_dataset.word_idx['end'], ioi_dataset.s_tokenIDs]
    # Get logits for subject
    logit_diff = io_logits - s_logits
    return logit_diff if per_prompt else logit_diff.mean()

with t.no_grad():
    clean_logits = model(clean_dataset.toks)
    corrupt_logits = model(corr_dataset.toks)
    clean_logit_diff = ave_logit_diff(clean_logits, clean_dataset).item()
    # corrupt_logit_diff = ave_logit_diff(corrupt_logits, corr_dataset).item()
    corrupt_logit_diff = ave_logit_diff(corrupt_logits, corr_dataset).item()

def ioi_metric(
    logits: Float[Tensor, "batch seq_len d_vocab"],
    corrupted_logit_diff: float = corrupt_logit_diff,
    clean_logit_diff: float = clean_logit_diff,
    ioi_dataset: IOIDataset = clean_dataset
 ):
    patched_logit_diff = ave_logit_diff(logits, ioi_dataset)
    return (patched_logit_diff - corrupted_logit_diff) / (clean_logit_diff - corrupted_logit_diff)

def abs_ioi_metric(logits: Float[Tensor, "batch seq_len d_vocab"]):
    return abs(ioi_metric(logits))

def negative_ioi_metric(logits: Float[Tensor, "batch seq_len d_vocab"]):
    return -ioi_metric(logits)

def negative_abs_ioi_metric(logits: Float[Tensor, "batch seq_len d_vocab"]):
    return -abs_ioi_metric(logits)

# Get clean and corrupt logit differences
with t.no_grad():
    clean_metric = ioi_metric(clean_logits, corrupt_logit_diff, clean_logit_diff, clean_dataset)
    corrupt_metric = ioi_metric(corrupt_logits, corrupt_logit_diff, clean_logit_diff, corr_dataset)

print(f'Clean direction: {clean_logit_diff}, Corrupt direction: {corrupt_logit_diff}')
print(f'Clean metric: {clean_metric}, Corrupt metric: {corrupt_metric}')

logits.shape=torch.Size([100, 16, 50257]), ioi_dataset.word_idx['end'].shape=torch.Size([100]), len(ioi_dataset.io_tokenIDs)=100
logits.shape=torch.Size([100, 16, 50257]), ioi_dataset.word_idx['end'].shape=torch.Size([100]), len(ioi_dataset.io_tokenIDs)=100
logits.shape=torch.Size([100, 16, 50257]), ioi_dataset.word_idx['end'].shape=torch.Size([100]), len(ioi_dataset.io_tokenIDs)=100
logits.shape=torch.Size([100, 16, 50257]), ioi_dataset.word_idx['end'].shape=torch.Size([100]), len(ioi_dataset.io_tokenIDs)=100
Clean direction: 3.131037473678589, Corrupt direction: 1.5243927240371704
Clean metric: 1.0, Corrupt metric: 0.0


In [6]:
print(corrupt_logit_diff)
print(clean_logit_diff)

1.5243927240371704
3.131037473678589


# Run Experiment

In [7]:
print(clean_dataset.toks.shape)
print(corr_dataset.toks.shape)

torch.Size([100, 16])
torch.Size([100, 16])


In [8]:
# import torch
# torch.save((clean_dataset.toks, corr_dataset.toks), "ioi_dataset_toks.pt")

In [9]:
from ACDCPPExperiment import ACDCPPExperiment
THRESHOLDS = [0.08, .15]#np.arange(0.005, 0.155, 0.005)
RUN_NAME = 'abs_edge'

acdcpp_exp = ACDCPPExperiment(
    model=model,
    clean_data=clean_dataset.toks,
    corr_data=corr_dataset.toks,
    acdc_metric=negative_abs_ioi_metric,
    acdcpp_metric=ioi_metric,
    thresholds=THRESHOLDS,
    run_name=RUN_NAME,
    verbose=False,
    attr_absolute_val=True,
    save_graphs_after=-100,
    pruning_mode='edge',
    no_pruned_nodes_attr=1,
    run_acdc=False,
    run_acdcpp=True,
)
# e=acdcpp_exp.setup_exp(0.0)

pruned_heads, num_passes, acdcpp_pruned_attrs, acdc_pruned_attrs, edges_after_acdcpp, edges_after_acdc = acdcpp_exp.run()

  0%|          | 0/2 [00:00<?, ?it/s]WARNING:root:cache_all is deprecated and will eventually be removed, use add_caching_hooks or run_with_cache


self.current_node=TLACDCInterpNode(blocks.11.hook_resid_post, [:])
logits.shape=torch.Size([100, 16, 50257]), ioi_dataset.word_idx['end'].shape=torch.Size([100]), len(ioi_dataset.io_tokenIDs)=100
logits.shape=torch.Size([100, 16, 50257]), ioi_dataset.word_idx['end'].shape=torch.Size([100]), len(ioi_dataset.io_tokenIDs)=100


 50%|█████     | 1/2 [00:11<00:11, 11.96s/it]

dict_keys([-1, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 12])


self.current_node=TLACDCInterpNode(blocks.11.hook_resid_post, [:])
logits.shape=torch.Size([100, 16, 50257]), ioi_dataset.word_idx['end'].shape=torch.Size([100]), len(ioi_dataset.io_tokenIDs)=100
logits.shape=torch.Size([100, 16, 50257]), ioi_dataset.word_idx['end'].shape=torch.Size([100]), len(ioi_dataset.io_tokenIDs)=100


100%|██████████| 2/2 [00:23<00:00, 11.76s/it]

dict_keys([-1, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 12])


In [10]:
len(edges_after_acdcpp[0.08])

179

In [8]:
len(edges_after_acdcpp[0.08])

227

In [ ]:
import pickle
with open(f"acdcpp_edges.pkl", "wb") as f:
    pickle.dump((edges_after_acdcpp, acdcpp_pruned_attrs), f)

In [ ]:
edges_after_acdcpp

{0.3: {'blocks.0.hook_mlp_in[:]blocks.0.hook_resid_pre[:]',
  'blocks.0.hook_mlp_out[:]blocks.0.hook_mlp_in[:]',
  'blocks.1.hook_mlp_out[:]blocks.1.hook_mlp_in[:]',
  'blocks.10.attn.hook_k[:, :, 7]blocks.10.hook_k_input[:, :, 7]',
  'blocks.10.attn.hook_q[:, :, 0]blocks.10.hook_q_input[:, :, 0]',
  'blocks.10.attn.hook_q[:, :, 6]blocks.10.hook_q_input[:, :, 6]',
  'blocks.10.attn.hook_q[:, :, 7]blocks.10.hook_q_input[:, :, 7]',
  'blocks.10.attn.hook_result[:, :, 0]blocks.10.attn.hook_k[:, :, 0]',
  'blocks.10.attn.hook_result[:, :, 0]blocks.10.attn.hook_q[:, :, 0]',
  'blocks.10.attn.hook_result[:, :, 0]blocks.10.attn.hook_v[:, :, 0]',
  'blocks.10.attn.hook_result[:, :, 7]blocks.10.attn.hook_k[:, :, 7]',
  'blocks.10.attn.hook_result[:, :, 7]blocks.10.attn.hook_q[:, :, 7]',
  'blocks.10.attn.hook_result[:, :, 7]blocks.10.attn.hook_v[:, :, 7]',
  'blocks.10.attn.hook_v[:, :, 0]blocks.10.hook_v_input[:, :, 0]',
  'blocks.10.attn.hook_v[:, :, 7]blocks.10.hook_v_input[:, :, 7]',
  'blo

# Save Data

In [ ]:
import json

for thresh in pruned_heads.keys():
    pruned_heads[thresh][0] = list(pruned_heads[thresh][0])
    pruned_heads[thresh][1] = list(pruned_heads[thresh][1])
    
cleaned_pp_attrs = {}
for thresh in acdcpp_pruned_attrs.keys():
    cleaned_pp_attrs[thresh] = {}
    for (parent, child), attr in acdcpp_pruned_attrs[thresh].items():
        cleaned_pp_attrs[thresh]\
        [f'{child.hook_point_name}{child.replace_parens(child.index)}{parent.hook_point_name}{parent.replace_parens(parent.index)}'] = attr
        
# Cleaning the edges
for thresh in edges_after_acdcpp.keys():
    edges_after_acdcpp[thresh] = list(edges_after_acdcpp[thresh])
    edges_after_acdc[thresh] = list(edges_after_acdc[thresh])
    
APPEND = False

if APPEND:
    with open(f'{RUN_NAME}_pruned_heads.json', 'r') as f:
        old_pruned_heads = json.load(f)
    with open(f'{RUN_NAME}_acdcpp_pruned_attrs.json', 'r') as f:
        old_acdcpp_pruned_attrs = json.load(f)
    with open(f'{RUN_NAME}_acdc_pruned_attrs.json', 'r') as f:
        old_acdc_pruned_attrs = json.load(f)
    with open(f'{RUN_NAME}_num_passes.json', 'r') as f:
        old_num_passes = json.load(f)
    with open(f'{RUN_NAME}_edges_after_acdcpp.json', 'r') as f:
        old_edges_after_acdcpp = json.load(f)
    with open(f'{RUN_NAME}_edges_after_acdc.json', 'r') as f:
        old_edges_after_acdc = json.load(f)
    
    pruned_heads = pruned_heads.update(old_pruned_heads)
    cleaned_pp_attrs = cleaned_pp_attrs.update(old_acdcpp_pruned_attrs)
    acdc_pruned_attrs = acdc_pruned_attrs.update(old_acdc_pruned_attrs)
    num_passes = num_passes.update(old_num_passes)
    edges_after_acdcpp = edges_after_acdcpp.update(old_edges_after_acdcpp)
    edges_after_acdc = edges_after_acdc.update(old_edges_after_acdc)

with open(f'{RUN_NAME}_pruned_heads.json', 'w') as f:
    json.dump(pruned_heads, f)
with open(f'{RUN_NAME}_num_passes.json', 'w') as f:
    json.dump(num_passes, f)
with open(f'{RUN_NAME}_acdcpp_pruned_attrs.json', 'w') as f:
    json.dump(cleaned_pp_attrs, f)
with open(f'{RUN_NAME}_acdc_pruned_attrs.json', 'w') as f:
    json.dump(acdc_pruned_attrs, f)

with open(f'{RUN_NAME}_edges_after_acdcpp.json', 'w') as f:
    json.dump(edges_after_acdcpp, f)
with open(f'{RUN_NAME}_edges_after_acdc.json', 'w') as f:
    json.dump(edges_after_acdc, f)

ValueError: too many values to unpack (expected 2)